# Neural Network from scratch.
    Answers are done in this notebook.

Imports

In [247]:
import numpy as np
import random

Defining the sigmoid funciton.

In [248]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

Defining the Sigmoid function with +ve z.

In [249]:
def sigmoid_pos(z):
    return 1.0/(1.0+np.exp(z))

Defining derivative of Sigmoid function.

In [250]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

Defining derivative of Sigmoid function with +ve z.

In [251]:
def sigmoid_pos_prime(z):
    """Derivative of the sigmoid function."""
    return -(sigmoid_pos(z)*(1-sigmoid_pos(z)))

Create Network class for neural network.

In [260]:

class Network():

    def __init__(self, sizes, activation_function=sigmoid, activation_function_prime=sigmoid_prime):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        # Take the activation function as an argument and initialize it as a class method
        self.activation_function = activation_function
        self.activation_function_prime = activation_function_prime

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = self.activation_function(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs. If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            cost_arr = [] # Keep track of the cost for each epoch
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                cost_arr.append(self.cost(mini_batch))
            if test_data:
                test_eval = self.evaluate(test_data)
                print(
                    f"Epoch {j}: {test_eval} / {n_test} Accuracy: {round(100*(test_eval/n_test), 2)} Cost: {np.mean(cost_arr)}"
                )
            else:
                print(f"Epoch {j} complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                        for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = self.activation_function(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            self.activation_function_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.activation_function_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost(self, mini_batch):
            """Return the cost of the network for each epoch"""
            square_error = [sum((y - self.feedforward(x))**2)
                            for x, y in mini_batch]
            return np.mean(square_error) / 2
                
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives (partial C_x) /
        (partial y) for the output activations."""
        return (output_activations-y)

Loading data using mnist_loader tool.

In [253]:
from tools import mnist_loader
# take only train_data and test_data
training_data, _, test_data = mnist_loader.load_data_wrapper()

Initializing network object with sigmoid activation function.

In [254]:
nn_with_sigmoid = Network(
                [784, 30, 10],activation_function=sigmoid,
                activation_function_prime=sigmoid_prime
                )

Training

In [255]:
nn_with_sigmoid.SGD(training_data, 10, 10, 3.0, test_data=test_data)

Epoch 0: 8967 / 10000 Cost: 0.1364484121219627
Epoch 1: 9128 / 10000 Cost: 0.047801207277343306
Epoch 2: 9250 / 10000 Cost: 0.03999780601801576
Epoch 3: 9337 / 10000 Cost: 0.035721526145789405
Epoch 4: 9318 / 10000 Cost: 0.033039261803944384
Epoch 5: 9372 / 10000 Cost: 0.03137646785808741
Epoch 6: 9403 / 10000 Cost: 0.029611429265433344
Epoch 7: 9419 / 10000 Cost: 0.027867557634382183
Epoch 8: 9430 / 10000 Cost: 0.0271872223527151
Epoch 9: 9455 / 10000 Cost: 0.02605476967860194


### Answer:
    The training went well. The Accuracy is 94.5%.
    Improvements:
    - Initalizing the weignts using glorot method.
    - Using cross validation to find the best hyperparameters.
    
    

Initializing network object with sigmoid_pos activation function.

In [256]:
nn_with_sigmoid_pos = Network(
                [784, 30, 10],activation_function=sigmoid_pos,
                activation_function_prime=sigmoid_pos_prime
                )

Training.

In [257]:
nn_with_sigmoid_pos.SGD(training_data, 10, 10, 3.0, test_data=test_data)

Epoch 0: 9060 / 10000 Cost: 0.08996456812940505
Epoch 1: 9242 / 10000 Cost: 0.04363640478727131
Epoch 2: 9287 / 10000 Cost: 0.03719470566175102
Epoch 3: 9320 / 10000 Cost: 0.03330797622406372
Epoch 4: 9333 / 10000 Cost: 0.031236756681594993
Epoch 5: 9362 / 10000 Cost: 0.02979664380250648
Epoch 6: 9370 / 10000 Cost: 0.02814211668519065
Epoch 7: 9354 / 10000 Cost: 0.026825489435866597
Epoch 8: 9393 / 10000 Cost: 0.0261362489620615
Epoch 9: 9372 / 10000 Cost: 0.02538049501368516


### Answer:
    Yes, it works for sigmoid activation function with +ve z.

Adding two more hidden layers.

In [258]:
nn_with_sigmoid = Network(
                [784, 30, 30, 30, 10],activation_function=sigmoid,
                activation_function_prime=sigmoid_prime
                )

In [259]:
nn_with_sigmoid.SGD(training_data, 10, 10, 3.0, test_data=test_data)

Epoch 0: 8940 / 10000 Cost: 0.11184793591720718
Epoch 1: 9185 / 10000 Cost: 0.04192322945040213
Epoch 2: 9208 / 10000 Cost: 0.03626577693455503
Epoch 3: 9262 / 10000 Cost: 0.03247335363704397
Epoch 4: 9291 / 10000 Cost: 0.030083227173701134
Epoch 5: 9312 / 10000 Cost: 0.028361085598474063
Epoch 6: 9372 / 10000 Cost: 0.026856454316355985
Epoch 7: 9239 / 10000 Cost: 0.025839068924285696
Epoch 8: 9376 / 10000 Cost: 0.024770992958933744
Epoch 9: 9384 / 10000 Cost: 0.02316413131424503


### Answer:
    With more layer the accuracy doesn't improved much. This might be due to random selection of number and size of the hidden layers.